In [1]:
import pandas as pd
import pandas_gbq
import plotly.express as px
import pandas_gbq
import sys, requests
from pathlib import Path
from zipfile import ZipFile
import urllib
from PIL import Image
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option("styler.format.thousands", ",")

In [2]:
#Change the date to the previous day
query = """
SELECT * FROM `perceptive-ivy-290216.nba_api_team_boxscore.team_box_96_onwards`
"""
project_id = "perceptive-ivy-290216"
df_bq_reg_season = pandas_gbq.read_gbq(query, project_id=project_id, dialect='standard')

Downloading: 100%|██████████|


In [49]:
#USE FOR ALL TIME DATA
# df_bq2=df_bq_reg_season.copy()

#USE FOR SINGLE SEASON PLOTS
current_year="2023-24"
df_bq2=df_bq_reg_season[df_bq_reg_season["Season"]==current_year].copy()

df_bq2=df_bq2.fillna(0)
df_bq2.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE,Season
0,22023,1610612737,ATL,Atlanta Hawks,0022300063,2023-10-25,ATL @ CHA,L,240,39,93,0.419,5,29,0.172,27,33,0.818,12,30,42,24,12,1,12,19,110,-6,1,2023-24
1,22023,1610612737,ATL,Atlanta Hawks,0022300533,2024-01-12,ATL vs. IND,L,240,40,89,0.449,11,30,0.367,17,23,0.739,12,16,28,25,14,2,15,13,108,-18,1,2023-24
2,22023,1610612737,ATL,Atlanta Hawks,0022300626,2024-01-24,ATL @ GSW,L,240,39,94,0.415,14,43,0.326,20,26,0.769,13,32,45,30,7,2,11,16,112,-22,1,2023-24
3,22023,1610612737,ATL,Atlanta Hawks,0022300384,2023-12-22,ATL @ MIA,L,240,42,96,0.438,19,54,0.352,10,14,0.714,12,31,43,29,9,2,14,23,113,-9,1,2023-24
4,22023,1610612737,ATL,Atlanta Hawks,0022300296,2023-12-11,ATL vs. DEN,L,240,44,96,0.458,17,44,0.386,17,18,0.944,13,22,35,25,9,2,10,21,122,-7,1,2023-24


In [23]:
def team_color(team):
    if team =="ATL":
        return "#C8102E"
    elif team =="DAL":
        return '#0050B5'
    elif team =="MEM":
        return '#0C2340'
    elif team =="MIA":
        return '#862633'
    elif team =="DEN":
        return '#0C2340'
    elif team =="DET":
        return '#1D4289'
    elif team =="GSW":
        return '#1D4289'
    elif team =="HOU":
        return '#BA0C2F'
    elif team =="IND":
        return '#041E42'
    elif team =="LAC":
        return '#0C2340'
    elif team =="LAL":
        return '#6a00ed'
    elif team =="POR":
        return '#C8102E'
    elif team =="PHX":
        return '#e56020'
    elif team =="PHI":
        return '#1D4289'
    elif team =="ORL":
        return '#0057B7'
    elif team =="OKC":
        return '#0072CE'
    elif team =="NYK":
        return '#FF8200'
    elif team =="NOP":
        return '#0C2340'
    elif team =="MIN":
        return '#0C2340'
    elif team =="MIL":
        return '#2C5234'
    elif team =="BKN":
        return '#707372'
    elif team =="BOS":
        return '#007A33'
    elif team =="SEA":
        return '#00653A'
    elif team =="CHA":
        return '#00778B'
    elif team =="BUF":
        return '#418FDE'
    elif team =="CHI":
        return '#BA0C2F'       
    elif team =="CLE":
        return '#6F263D'
    elif team =="WAS":
        return '#C8102E'
    elif team =="UTA":
        return '#0C2340' 
    elif team =="TOR":
        return '#C8102E'
    elif team =="SAS":
        return '#9EA2A2'
    elif team =="SAC":
        return '#582C83'                  
    else:
        return '#3860C6'

In [55]:
df_bq2['color'] = df_bq2['TEAM_ABBREVIATION'].apply(team_color)
team_color_dict=dict(zip(df_bq2['TEAM_ABBREVIATION'], df_bq2['color']))

In [56]:
df_bq2['FGMX']=df_bq2['FGA']-df_bq2['FGM']
df_bq2['FG3MX']=df_bq2['FG3A']-df_bq2['FG3M']
df_bq2['FG2M']=df_bq2['FGM']-df_bq2['FG3M']
df_bq2['FG2A']=df_bq2['FGA']-df_bq2['FG3A']
df_bq2['FG2_PCT']=df_bq2['FG2M']/df_bq2['FG2A']
df_bq2=df_bq2.round({'FG2_PCT': 3})

In [57]:
#Add Game Number for trended values plotting
df_bq2["Game Number"] = df_bq2.groupby("TEAM_NAME")["GAME_DATE"].rank(method="dense", ascending=True)
df_bq2["Game Number"]=df_bq2["Game Number"].astype(int)

In [58]:
#Get cumulative stat for each player
df_bq2['Total PTS'] = df_bq2.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['PTS'].cumsum()
df_bq2['Total REB'] = df_bq2.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['REB'].cumsum()
df_bq2['Total DREB'] = df_bq2.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['DREB'].cumsum()
df_bq2['Total OREB'] = df_bq2.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['OREB'].cumsum()
df_bq2['Total AST'] = df_bq2.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['AST'].cumsum()
df_bq2['Total STL'] = df_bq2.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['STL'].cumsum()
df_bq2['Total BLK'] = df_bq2.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['BLK'].cumsum()
df_bq2['Total TOV'] = df_bq2.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['TOV'].cumsum()
df_bq2['Total PF'] = df_bq2.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['PF'].cumsum()
df_bq2['Total FGM'] = df_bq2.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['FGM'].cumsum()
df_bq2['Total FGA'] = df_bq2.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['FGA'].cumsum()
df_bq2['Total FG3M'] = df_bq2.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['FG3M'].cumsum()
df_bq2['Total FG3A'] = df_bq2.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['FG3A'].cumsum()
df_bq2['Total FGMX'] = df_bq2.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['FGMX'].cumsum()
df_bq2['Total FG3MX'] = df_bq2.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['FG3MX'].cumsum()
df_bq2['Total FTM'] = df_bq2.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['FTM'].cumsum()
df_bq2['Total FTA'] = df_bq2.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['FTA'].cumsum()
df_bq2['Total MIN'] = df_bq2.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['MIN'].cumsum()

In [59]:
#Get Totals For Each Category
df_totals=pd.DataFrame(df_bq2.groupby(by=['TEAM_NAME','TEAM_ABBREVIATION'])[['MIN', 'FGM',
       'FGA', 'FG3M', 'FG3A', 'FTM', 'FTA','OREB', 'DREB', 'REB', 'AST',
       'STL', 'BLK', 'TOV', 'PF', 'PTS','PLUS_MINUS','FGMX','FG3MX','FG2M','FG2A']].sum()).reset_index()
#Get Game Counts for Each Player
df_count=pd.DataFrame(df_bq2.groupby(by=['TEAM_NAME'])[["GAME_ID"]].count()).reset_index()
df_count.columns=["TEAM_NAME","GP"]
df_totals=df_totals.merge(df_count, left_on=["TEAM_NAME"],right_on=["TEAM_NAME"])
df_totals.head()

,TEAM_NAME,TEAM_ABBREVIATION,MIN,FGM,FGA,FG3M,FG3A,FTM,FTA,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,FGMX,FG3MX,FG2M,FG2A,GP
0,Atlanta Hawks,ATL,19855,3529,7584,1125,3092,1520,1906,1024,2639,3663,2180,616,369,1110,1522,9703,-179,4055,1967,2404,4492,82
1,Boston Celtics,BOS,19830,3601,7396,1351,3482,1334,1654,876,2923,3799,2207,557,538,979,1326,9887,930,3795,2131,2250,3914,82
2,Brooklyn Nets,BKN,19805,3334,7307,1089,3010,1293,1711,938,2675,3613,2102,556,424,1076,1516,9050,-237,3973,1921,2245,4297,82
3,Charlotte Hornets,CHA,19730,3281,7133,989,2788,1189,1512,765,2538,3303,2033,562,371,1129,1472,8740,-840,3852,1799,2292,4345,82
4,Chicago Bulls,CHI,19980,3448,7339,941,2630,1369,1731,916,2677,3593,2048,638,394,1004,1541,9206,-118,3891,1689,2507,4709,82


In [60]:
#Add additional columns
df_totals['FG_PCT']=df_totals['FGM']/df_totals['FGA']
df_totals['FG3_PCT']=df_totals['FG3M']/df_totals['FG3A']
df_totals['FG2_PCT']=df_totals['FG2M']/df_totals['FG2A']
df_totals=df_totals.round({'FG_PCT': 3,'FG3_PCT': 3,'FG2_PCT': 3})

In [61]:
#Re-order the columns
df_totals=df_totals[['TEAM_NAME','TEAM_ABBREVIATION', 'GP', 'MIN', 'FGM', 'FGMX', 'FGA','FG_PCT', 'FG2M', 'FG2A','FG2_PCT','FG3M', 
          'FG3MX', 'FG3A', 'FG3_PCT', 'FTM', 'FTA','OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS']]
# df_totals["MIN"]=df_totals["MIN"].apply(lambda x : "{:,}".format(x))
df_totals.head()

,TEAM_NAME,TEAM_ABBREVIATION,GP,MIN,FGM,FGMX,FGA,FG_PCT,FG2M,FG2A,FG2_PCT,FG3M,FG3MX,FG3A,FG3_PCT,FTM,FTA,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS
0,Atlanta Hawks,ATL,82,19855,3529,4055,7584,0.465,2404,4492,0.535,1125,1967,3092,0.364,1520,1906,1024,2639,3663,2180,616,369,1110,1522,9703,-179
1,Boston Celtics,BOS,82,19830,3601,3795,7396,0.487,2250,3914,0.575,1351,2131,3482,0.388,1334,1654,876,2923,3799,2207,557,538,979,1326,9887,930
2,Brooklyn Nets,BKN,82,19805,3334,3973,7307,0.456,2245,4297,0.522,1089,1921,3010,0.362,1293,1711,938,2675,3613,2102,556,424,1076,1516,9050,-237
3,Charlotte Hornets,CHA,82,19730,3281,3852,7133,0.46,2292,4345,0.528,989,1799,2788,0.355,1189,1512,765,2538,3303,2033,562,371,1129,1472,8740,-840
4,Chicago Bulls,CHI,82,19980,3448,3891,7339,0.47,2507,4709,0.532,941,1689,2630,0.358,1369,1731,916,2677,3593,2048,638,394,1004,1541,9206,-118


In [72]:
sort_category="FG3M"

In [73]:
#Sort by a category and then rank it. This will help get the Top Players in a category
df_totals['RK_Category']=df_totals[sort_category].rank(method="dense", ascending=False)
df_totals['RK_Category']=df_totals['RK_Category'].astype(int)
top=df_totals.sort_values(by=sort_category, ascending=False).head(10)
top

,TEAM_NAME,TEAM_ABBREVIATION,GP,MIN,FGM,FGMX,FGA,FG_PCT,FG2M,FG2A,FG2_PCT,FG3M,FG3MX,FG3A,FG3_PCT,FTM,FTA,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,RK_Category
1,Boston Celtics,BOS,82,19830,3601,3795,7396,0.487,2250,3914,0.575,1351,2131,3482,0.388,1334,1654,876,2923,3799,2207,557,538,979,1326,9887,930,1
9,Golden State Warriors,GSW,82,19830,3582,3933,7515,0.477,2371,4324,0.548,1211,1980,3191,0.38,1282,1644,996,2834,3830,2402,572,377,1174,1598,9657,214,2
6,Dallas Mavericks,DAL,82,19705,3535,3817,7352,0.481,2338,4110,0.569,1197,2045,3242,0.369,1397,1842,797,2724,3521,2106,563,406,1027,1500,9664,181,3
25,Sacramento Kings,SAC,82,19855,3553,3902,7455,0.477,2375,4236,0.561,1178,2041,3219,0.366,1274,1710,888,2719,3607,2324,626,342,1078,1632,9558,143,4
16,Milwaukee Bucks,MIL,82,19805,3538,3720,7258,0.487,2375,4136,0.574,1163,1959,3122,0.373,1517,1960,772,2850,3622,2176,558,413,1059,1574,9756,215,5
0,Atlanta Hawks,ATL,82,19855,3529,4055,7584,0.465,2404,4492,0.535,1125,1967,3092,0.364,1520,1906,1024,2639,3663,2180,616,369,1110,1522,9703,-179,6
5,Cleveland Cavaliers,CLE,82,19805,3425,3723,7148,0.479,2317,4132,0.561,1108,1908,3016,0.367,1278,1671,807,2742,3549,2299,605,380,1112,1436,9236,198,7
20,Oklahoma City Thunder,OKC,82,19805,3653,3671,7324,0.499,2563,4519,0.567,1090,1715,2805,0.389,1451,1759,722,2725,3447,2223,694,538,1039,1545,9847,608,8
2,Brooklyn Nets,BKN,82,19805,3334,3973,7307,0.456,2245,4297,0.522,1089,1921,3010,0.362,1293,1711,938,2675,3613,2102,556,424,1076,1516,9050,-237,9
19,New York Knicks,NYK,82,19730,3385,3887,7272,0.465,2302,4336,0.531,1083,1853,2936,0.369,1396,1789,1039,2665,3704,1998,612,336,1085,1447,9249,376,10


In [74]:
#Get Player Names of the Top Category we have selected
names=top.TEAM_NAME.to_list()

In [75]:
#Get the DF with Player Names from the top category
df_top=df_bq2[df_bq2['TEAM_NAME'].isin(names)]

In [76]:
#Get Trended Data Point 

In [77]:
fig = px.scatter(
  df_top, 
  x="Game Number", 
  y="Total {}".format(sort_category),
  color='TEAM_ABBREVIATION',
color_discrete_map=team_color_dict,     
title="<b>Regular Season All Time Trended Player {} vs. Game Number</b>".format(sort_category),
  template="plotly_white",
  hover_data=['TEAM_NAME', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'OREB',
       'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS', 'Season',
       'Game Number'], 
  height=800, 
  width=1200,
  )

fig.update_layout(
    title_x=0.5,
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=20)),
    xaxis = dict(tickfont = dict(size=20)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig.update_traces(marker={'size': 3})

fig

In [ ]:
save_year=2024

In [ ]:
fig.write_html("/Users/rdesh723/statpulse-html/plots/Team Box Scores/{}/Regular Season All Time Trended Team {} vs. Game Number.html".format(save_year, sort_category),full_html=False, include_plotlyjs='cdn')
# fig.write_html("/Users/rdesh723/statpulse-html/plots/Team Box Scores/{}/Trended Team {} vs. Game Number For The {} Season.html".format(save_year, sort_category,current_year),full_html=False, include_plotlyjs='cdn')

In [230]:
#POLAR CHARTS BY TOTALS

In [78]:
polar=px.bar_polar(
top, 
r=sort_category, 
theta="TEAM_NAME", 
color="TEAM_ABBREVIATION",
template='ggplot2',
color_discrete_map=team_color_dict,     
 title="Regular Season All Time Leaders In Team Total {}".format(sort_category),
 hover_data=['TEAM_NAME', 'MIN', 'FGM', 'FGMX', 'FGA', 'FG_PCT', 'FG2M', 'FG2A',
       'FG2_PCT', 'FG3M', 'FG3MX', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'OREB',
       'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS',
       'RK_Category'],
 height=800, width=1200,
                  )
polar.update_layout(
    title_x=0.5,
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=20)),
    xaxis = dict(tickfont = dict(size=20)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)

In [ ]:
polar.write_html("/Users/rdesh723/statpulse-html/plots/Team Box Scores/{}/Regular Season All Time Leaders Team In Total {}.html".format(save_year,sort_category),full_html=False, include_plotlyjs='cdn')
# polar.write_html("/Users/rdesh723/statpulse-html/plots/Team Box Scores/{}/Team Leaders In Total {} For The {} Season.html".format(save_year,sort_category, current_year),full_html=False, include_plotlyjs='cdn')

In [ ]:
#2 variable scatter plot

In [81]:
cat1="PTS"
cat2="AST"

fig_scatter = px.scatter(
  df_totals, 
  x=cat1, 
  y=cat2,
  color='TEAM_ABBREVIATION',
color_discrete_map=team_color_dict,     
  title="<b>Regular Season All Time Team {} vs. {}</b>".format(cat1,cat2),
  template="plotly_white",
  hover_data=['TEAM_NAME', 'GP', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'OREB',
       'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS'], 
  height=800, 
  width=1200,
  )

fig_scatter.update_layout(
    title_x=0.5,
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=20)),
    xaxis = dict(tickfont = dict(size=20)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig_scatter.update_traces(marker={'size': 10})

fig_scatter

In [ ]:
fig_scatter.write_html("/Users/rdesh723/statpulse-html/plots/Team Box Scores/{}/Regular Season All Time Team {} vs. {}.html".format(save_year,cat1,cat2),full_html=False, include_plotlyjs='cdn')
# fig_scatter.write_html("/Users/rdesh723/statpulse-html/plots/Team Box Scores/{}/Team {} vs. {} For The {} Season.html".format(save_year,cat1,cat2, current_year),full_html=False, include_plotlyjs='cdn')

In [ ]:
#MULTI METRIC PLOTS TOGETHER

In [50]:
sort_categories=['PTS', 'REB','DREB', 'OREB', 'AST', 'STL', 'BLK','TOV', 'PF', 'FGM', 'FGA', 'FG3M',
       'FG3A', 'FGMX', 'FG3MX', 'FTM',  'FTA','MIN']

In [51]:
#TRENDED

In [53]:
df_top.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE,Season
0,22023,1610612737,ATL,Atlanta Hawks,0022300063,2023-10-25,ATL @ CHA,L,240,39,93,0.419,5,29,0.172,27,33,0.818,12,30,42,24,12,1,12,19,110,-6,1,2023-24
1,22023,1610612737,ATL,Atlanta Hawks,0022300533,2024-01-12,ATL vs. IND,L,240,40,89,0.449,11,30,0.367,17,23,0.739,12,16,28,25,14,2,15,13,108,-18,1,2023-24
2,22023,1610612737,ATL,Atlanta Hawks,0022300626,2024-01-24,ATL @ GSW,L,240,39,94,0.415,14,43,0.326,20,26,0.769,13,32,45,30,7,2,11,16,112,-22,1,2023-24
3,22023,1610612737,ATL,Atlanta Hawks,0022300384,2023-12-22,ATL @ MIA,L,240,42,96,0.438,19,54,0.352,10,14,0.714,12,31,43,29,9,2,14,23,113,-9,1,2023-24
4,22023,1610612737,ATL,Atlanta Hawks,0022300296,2023-12-11,ATL vs. DEN,L,240,44,96,0.458,17,44,0.386,17,18,0.944,13,22,35,25,9,2,10,21,122,-7,1,2023-24


In [69]:
for i in sort_categories:
    df_totals['RK_Category']=df_totals[i].rank(method="dense", ascending=False)
    df_totals['RK_Category']=df_totals['RK_Category'].astype(int)
    top=df_totals.sort_values(by=i, ascending=False).head(10)
    names=top.TEAM_NAME.to_list()
    df_top=df_bq2[df_bq2['TEAM_NAME'].isin(names)]

    fig_multi = px.scatter(
    df_top, 
    x="Game Number", 
    y="Total {}".format(i),
    color='TEAM_ABBREVIATION',
    color_discrete_map=team_color_dict,     
    title="<b>Regular Season All Time Trended Team {} vs. Game Number</b>".format(i),
    template="plotly_white",
    hover_data=['TEAM_ABBREVIATION','Total PTS', 'Total REB',
       'Total DREB', 'Total OREB', 'Total AST', 'Total STL', 'Total BLK',
       'Total TOV', 'Total PF', 'Total FGM', 'Total FGA', 'Total FG3M',
       'Total FG3A', 'Total FGMX', 'Total FG3MX', 'Total FTM', 'Total FTA',
       'Total MIN'
        #          'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'OREB',
        # 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS', 'Season',
        # 'Game Number'
        ], 
    height=800, 
    width=1200,
    )

    fig_multi.update_layout(
        title_x=0.5,
        hoverlabel=dict(
            bgcolor="white",
            font_size=16,
            font_family="PT Sans Narrow"
        ),
        yaxis = dict(tickfont = dict(size=20)),
        xaxis = dict(tickfont = dict(size=20)),
        font=dict(
            family="PT Sans Narrow",
            size=14,
            color="Black"
        ),
        title_font_family="PT Sans Narrow"
    )
    fig_multi.update_traces(marker={'size': 3})
    fig_multi.write_html("/Users/rdesh723/statpulse-html/plots/Team Box Scores/{}/Regular Season All Time Trended Team {} vs. Game Number.html".format(save_year,i),full_html=False, include_plotlyjs='cdn')
    # fig_multi.write_html("/Users/rdesh723/statpulse-html/plots/Team Box Scores/{}/Trended Team {} vs. Game Number For The {} Season.html".format(save_year,i, current_year),full_html=False, include_plotlyjs='cdn')    

In [ ]:
#POLAR

In [84]:
for i in sort_categories:
    df_totals['RK_Category']=df_totals[i].rank(method="dense", ascending=False)
    df_totals['RK_Category']=df_totals['RK_Category'].astype(int)
    top=df_totals.sort_values(by=i, ascending=False).head(10)
    names=top.TEAM_NAME.to_list()
    df_top=df_bq2[df_bq2['TEAM_NAME'].isin(names)]
    
    polar_multi=px.bar_polar(
    top, 
    r=i, 
    theta="TEAM_NAME", 
    color="TEAM_ABBREVIATION",
    template='ggplot2',
    color_discrete_map=team_color_dict,     
    # title="Regular Season All Time Leaders In Total {}".format(i),
    title="Leaders In Total Team {} For The {} Season".format(i, current_year),
    hover_data=['TEAM_NAME', 'MIN', 'FGM', 'FGMX', 'FGA', 'FG_PCT', 'FG2M', 'FG2A',
        'FG2_PCT', 'FG3M', 'FG3MX', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'OREB',
        'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS',
        'RK_Category'],
    height=800, width=1200,
                    )
    polar_multi.update_layout(
        title_x=0.5,
        hoverlabel=dict(
            bgcolor="white",
            font_size=16,
            font_family="PT Sans Narrow"
        ),
        yaxis = dict(tickfont = dict(size=20)),
        xaxis = dict(tickfont = dict(size=20)),
        font=dict(
            family="PT Sans Narrow",
            size=14,
            color="Black"
        ),
        title_font_family="PT Sans Narrow"
    )
    # polar_multi.write_html("/Users/rdesh723/statpulse-html/plots/Team Box Scores/{}/Regular Season All Time Leaders In Total Team {}.html".format(save_year,i),full_html=False, include_plotlyjs='cdn')
    polar_multi.write_html("/Users/rdesh723/statpulse-html/plots/Team Box Scores/{}/Leaders In Total Team {} For The {} Season.html".format(save_year,i, current_year),full_html=False, include_plotlyjs='cdn')    

In [ ]:
# PLAYOFFS

In [85]:
#Change the date to the previous day
query = """
SELECT * FROM `perceptive-ivy-290216.nba_api_team_boxscore.playoffs_team_box_96_onwards`
"""
project_id = "perceptive-ivy-290216"
df_bq_playoffs = pandas_gbq.read_gbq(query, project_id=project_id, dialect='standard')

Downloading: 100%|██████████|


In [121]:
#USE FOR ALL TIME DATA
# df_bq2_playoffs=df_bq_playoffs.copy()

#USE FOR SINGLE SEASON PLOTS
current_year_playoffs='2023-24'
df_bq2_playoffs=df_bq_playoffs[df_bq_playoffs["Season"]==current_year_playoffs].copy()

df_bq2_playoffs=df_bq2_playoffs.fillna(0)
df_bq2_playoffs.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE,Season
345,42023,1610612738,BOS,Boston Celtics,0042300202,2024-05-09,BOS vs. CLE,L,240,33,80,0.413,8,35,0.229,20,24,0.833,6,25,31,22,5,3,7,15,94,-24,1,2023-24
346,42023,1610612738,BOS,Boston Celtics,0042300102,2024-04-24,BOS vs. MIA,L,240,37,80,0.463,12,32,0.375,15,21,0.714,6,33,39,21,9,8,13,17,101,-10,1,2023-24
347,42023,1610612738,BOS,Boston Celtics,0042300404,2024-06-14,BOS @ DAL,L,240,29,80,0.363,14,41,0.341,12,13,0.923,4,27,31,18,2,5,14,19,84,-38,1,2023-24
599,42023,1610612739,CLE,Cleveland Cavaliers,0042300134,2024-04-27,CLE @ ORL,L,240,34,72,0.472,4,17,0.235,17,21,0.810,2,27,29,24,10,3,15,20,89,-23,1,2023-24
600,42023,1610612739,CLE,Cleveland Cavaliers,0042300133,2024-04-25,CLE @ ORL,L,240,32,82,0.390,8,34,0.235,11,13,0.846,5,27,32,24,2,8,14,16,83,-38,1,2023-24


In [122]:
df_bq2_playoffs['FGMX']=df_bq2_playoffs['FGA']-df_bq2_playoffs['FGM']
df_bq2_playoffs['FG3MX']=df_bq2_playoffs['FG3A']-df_bq2_playoffs['FG3M']
df_bq2_playoffs['FG2M']=df_bq2_playoffs['FGM']-df_bq2_playoffs['FG3M']
df_bq2_playoffs['FG2A']=df_bq2_playoffs['FGA']-df_bq2_playoffs['FG3A']
df_bq2_playoffs['FG2_PCT']=df_bq2_playoffs['FG2M']/df_bq2_playoffs['FG2A']
df_bq2_playoffs=df_bq2_playoffs.round({'FG2_PCT': 3})

In [123]:
#Add Game Number for trended values plotting
df_bq2_playoffs["Game Number"] = df_bq2_playoffs.groupby("TEAM_NAME")["GAME_DATE"].rank(method="dense", ascending=True)
df_bq2_playoffs["Game Number"]=df_bq2_playoffs["Game Number"].astype(int)

In [124]:
#Get cumulative stat for each player
df_bq2_playoffs['Total PTS'] = df_bq2_playoffs.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['PTS'].cumsum()
df_bq2_playoffs['Total REB'] = df_bq2_playoffs.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['REB'].cumsum()
df_bq2_playoffs['Total DREB'] = df_bq2_playoffs.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['DREB'].cumsum()
df_bq2_playoffs['Total OREB'] = df_bq2_playoffs.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['OREB'].cumsum()
df_bq2_playoffs['Total AST'] = df_bq2_playoffs.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['AST'].cumsum()
df_bq2_playoffs['Total STL'] = df_bq2_playoffs.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['STL'].cumsum()
df_bq2_playoffs['Total BLK'] = df_bq2_playoffs.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['BLK'].cumsum()
df_bq2_playoffs['Total TOV'] = df_bq2_playoffs.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['TOV'].cumsum()
df_bq2_playoffs['Total PF'] = df_bq2_playoffs.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['PF'].cumsum()
df_bq2_playoffs['Total FGM'] = df_bq2_playoffs.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['FGM'].cumsum()
df_bq2_playoffs['Total FGA'] = df_bq2_playoffs.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['FGA'].cumsum()
df_bq2_playoffs['Total FG3M'] = df_bq2_playoffs.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['FG3M'].cumsum()
df_bq2_playoffs['Total FG3A'] = df_bq2_playoffs.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['FG3A'].cumsum()
df_bq2_playoffs['Total FGMX'] = df_bq2_playoffs.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['FGMX'].cumsum()
df_bq2_playoffs['Total FG3MX'] = df_bq2_playoffs.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['FG3MX'].cumsum()
df_bq2_playoffs['Total FTM'] = df_bq2_playoffs.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['FTM'].cumsum()
df_bq2_playoffs['Total FTA'] = df_bq2_playoffs.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['FTA'].cumsum()
df_bq2_playoffs['Total MIN'] = df_bq2_playoffs.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['MIN'].cumsum()

In [126]:
#Get Totals For Each Category
df_totals_playoffs=pd.DataFrame(df_bq2_playoffs.groupby(by=['TEAM_NAME','TEAM_ABBREVIATION'])[['MIN', 'FGM',
       'FGA', 'FG3M', 'FG3A', 'FTM', 'FTA','OREB', 'DREB', 'REB', 'AST',
       'STL', 'BLK', 'TOV', 'PF', 'PTS','PLUS_MINUS','FGMX','FG3MX','FG2M','FG2A']].sum()).reset_index()
#Get Game Counts for Each Player
df_count_playoffs=pd.DataFrame(df_bq2_playoffs.groupby(by=['TEAM_NAME'])[["GAME_ID"]].count()).reset_index()
df_count_playoffs.columns=["TEAM_NAME","GP"]
df_totals_playoffs=df_totals_playoffs.merge(df_count_playoffs, left_on=["TEAM_NAME"],right_on=["TEAM_NAME"])
df_totals_playoffs.head()

,TEAM_NAME,TEAM_ABBREVIATION,MIN,FGM,FGA,FG3M,FG3A,FTM,FTA,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,FGMX,FG3MX,FG2M,FG2A,GP
0,Boston Celtics,BOS,4585,747,1593,275,764,298,367,184,634,818,452,119,102,218,298,2067,153,846,489,472,829,19
1,Cleveland Cavaliers,CLE,2880,449,1001,124,394,155,204,92,373,465,255,80,55,146,237,1177,-67,552,270,325,607,12
2,Dallas Mavericks,DAL,5280,855,1827,262,728,359,499,239,709,948,465,138,116,276,417,2331,57,972,466,593,1099,22
3,Denver Nuggets,DEN,2880,467,1022,126,379,166,200,119,391,510,299,77,51,144,216,1226,-25,555,253,341,643,12
4,Indiana Pacers,IND,4130,761,1502,208,559,206,276,175,528,703,508,93,82,204,348,1936,18,741,351,553,943,17


In [127]:
#Add additional columns
df_totals_playoffs['FG_PCT']=df_totals_playoffs['FGM']/df_totals_playoffs['FGA']
df_totals_playoffs['FG3_PCT']=df_totals_playoffs['FG3M']/df_totals_playoffs['FG3A']
df_totals_playoffs['FG2_PCT']=df_totals_playoffs['FG2M']/df_totals_playoffs['FG2A']
df_totals_playoffs=df_totals_playoffs.round({'FG_PCT': 3,'FG3_PCT': 3,'FG2_PCT': 3})

In [128]:
#Re-order the columns
df_totals_playoffs=df_totals_playoffs[['TEAM_NAME','TEAM_ABBREVIATION', 'GP', 'MIN', 'FGM', 'FGMX', 'FGA','FG_PCT', 'FG2M', 'FG2A','FG2_PCT','FG3M', 
          'FG3MX', 'FG3A', 'FG3_PCT', 'FTM', 'FTA','OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS']]
# df_totals_playoffs["MIN"]=df_totals_playoffs["MIN"].apply(lambda x : "{:,}".format(x))
df_totals_playoffs.head()

,TEAM_NAME,TEAM_ABBREVIATION,GP,MIN,FGM,FGMX,FGA,FG_PCT,FG2M,FG2A,FG2_PCT,FG3M,FG3MX,FG3A,FG3_PCT,FTM,FTA,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS
0,Boston Celtics,BOS,19,4585,747,846,1593,0.469,472,829,0.569,275,489,764,0.36,298,367,184,634,818,452,119,102,218,298,2067,153
1,Cleveland Cavaliers,CLE,12,2880,449,552,1001,0.449,325,607,0.535,124,270,394,0.315,155,204,92,373,465,255,80,55,146,237,1177,-67
2,Dallas Mavericks,DAL,22,5280,855,972,1827,0.468,593,1099,0.54,262,466,728,0.36,359,499,239,709,948,465,138,116,276,417,2331,57
3,Denver Nuggets,DEN,12,2880,467,555,1022,0.457,341,643,0.53,126,253,379,0.332,166,200,119,391,510,299,77,51,144,216,1226,-25
4,Indiana Pacers,IND,17,4130,761,741,1502,0.507,553,943,0.586,208,351,559,0.372,206,276,175,528,703,508,93,82,204,348,1936,18


In [129]:
sort_category_playoffs="PTS"

In [130]:
#Sort by a category and then rank it. This will help get the Top Players in a category
df_totals_playoffs['RK_Category']=df_totals_playoffs[sort_category_playoffs].rank(method="dense", ascending=False)
df_totals_playoffs['RK_Category']=df_totals_playoffs['RK_Category'].astype(int)
top_playoffs=df_totals_playoffs.sort_values(by=sort_category_playoffs, ascending=False).head(10)
top_playoffs

,TEAM_NAME,TEAM_ABBREVIATION,GP,MIN,FGM,FGMX,FGA,FG_PCT,FG2M,FG2A,FG2_PCT,FG3M,FG3MX,FG3A,FG3_PCT,FTM,FTA,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,RK_Category
2,Dallas Mavericks,DAL,22,5280,855,972,1827,0.468,593,1099,0.54,262,466,728,0.36,359,499,239,709,948,465,138,116,276,417,2331,57,1
0,Boston Celtics,BOS,19,4585,747,846,1593,0.469,472,829,0.569,275,489,764,0.36,298,367,184,634,818,452,119,102,218,298,2067,153,2
4,Indiana Pacers,IND,17,4130,761,741,1502,0.507,553,943,0.586,208,351,559,0.372,206,276,175,528,703,508,93,82,204,348,1936,18,3
9,Minnesota Timberwolves,MIN,16,3840,620,706,1326,0.468,433,805,0.538,187,334,521,0.359,293,378,170,509,679,383,116,73,195,333,1720,67,4
11,New York Knicks,NYK,13,3145,517,625,1142,0.453,362,729,0.497,155,258,413,0.375,240,317,175,382,557,293,74,70,146,232,1429,-27,5
3,Denver Nuggets,DEN,12,2880,467,555,1022,0.457,341,643,0.53,126,253,379,0.332,166,200,119,391,510,299,77,51,144,216,1226,-25,6
1,Cleveland Cavaliers,CLE,12,2880,449,552,1001,0.449,325,607,0.535,124,270,394,0.315,155,204,92,373,465,255,80,55,146,237,1177,-67,7
12,Oklahoma City Thunder,OKC,10,2400,394,464,858,0.459,273,518,0.527,121,219,340,0.356,148,185,87,328,415,231,78,62,112,177,1057,63,8
13,Orlando Magic,ORL,7,1680,247,341,588,0.42,175,355,0.493,72,161,233,0.309,136,176,84,231,315,135,56,36,101,151,702,31,9
8,Milwaukee Bucks,MIL,6,1465,245,273,518,0.473,178,329,0.541,67,122,189,0.354,104,127,57,199,256,133,32,19,64,112,661,-17,10


In [131]:
#Get Player Names of the Top Category we have selected
names_playoffs=top_playoffs.TEAM_NAME.to_list()

In [132]:
#Get the DF with Player Names from the top category
df_top_playoffs=df_bq2_playoffs[df_bq2_playoffs['TEAM_NAME'].isin(names_playoffs)]

In [133]:
#Get Trended Data Point 

In [134]:
fig_playoffs = px.scatter(
  df_top_playoffs, 
  x="Game Number", 
  y="Total {}".format(sort_category_playoffs),
  color='TEAM_ABBREVIATION',
  color_discrete_map=team_color_dict,     
  title="<b>Playoffs All Time Trended Team {} vs. Game Number</b>".format(sort_category),
  title="Trended Team {} vs. Game Number For The {} Playoffs".format(sort_category_playoffs,current_year_playoffs), 
  template="plotly_white",
  hover_data=['TEAM_NAME', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'OREB',
       'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS', 'Season',
       'Game Number'], 
  height=800, 
  width=1200,
  )

fig_playoffs.update_layout(
    title_x=0.5,
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=20)),
    xaxis = dict(tickfont = dict(size=20)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig_playoffs.update_traces(marker={'size': 3})

fig_playoffs

In [ ]:
fig_playoffs.write_html("/Users/rdesh723/statpulse-html/plots/Team Box Scores/{}/Playoffs All Time Trended Team {} vs. Game Number.html".format(save_year,sort_category_playoffs),full_html=False, include_plotlyjs='cdn')
# fig_playoffs.write_html("/Users/rdesh723/statpulse-html/plots/Team Box Scores/{}/Trended Team {} vs. Game Number For The {} Playoffs.html".format(save_year,sort_category_playoffs,current_year_playoffs),full_html=False, include_plotlyjs='cdn')

In [ ]:
#POLAR CHARTS BY TOTALS

In [136]:
polar_playoffs=px.bar_polar(
top_playoffs, 
r=sort_category, 
theta="TEAM_NAME", 
color="TEAM_ABBREVIATION",
template='ggplot2',
color_discrete_map=team_color_dict,     
#  title="Playoffs All Time Leaders In Total {}".format(sort_category),
  title="Leaders In Total Team {} For The {} Playoffs".format(sort_category_playoffs, current_year_playoffs),
 hover_data=['TEAM_NAME', 'MIN', 'FGM', 'FGMX', 'FGA', 'FG_PCT', 'FG2M', 'FG2A',
       'FG2_PCT', 'FG3M', 'FG3MX', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'OREB',
       'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS',
       'RK_Category'],
 height=800, width=1200,
                  )
polar_playoffs.update_layout(
    title_x=0.5,
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=20)),
    xaxis = dict(tickfont = dict(size=20)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)

In [ ]:
polar_playoffs.write_html("/Users/rdesh723/statpulse-html/plots/Team Box Scores/{}/Playoffs All Time Leaders In Total Team {}.html".format(save_year,sort_category_playoffs),full_html=False, include_plotlyjs='cdn')
# polar_playoffs.write_html("/Users/rdesh723/statpulse-html/plots/Team Box Scores/{}/Leaders In Total Team {} For The {} Playoffs.html".format(save_year,sort_category_playoffs, current_year_playoffs),full_html=False, include_plotlyjs='cdn')

In [ ]:
#2 variable scatter plot

In [45]:
cat1_playoffs="GP"
cat2_playoffs="AST"

fig_scatter_playoffs = px.scatter(
  df_totals_playoffs, 
  x=cat1, 
  y=cat2,
  color='ABBREVIATIOM',
color_discrete_map=team_color_dict,     
  title="<b>Playoffs All Time Player {} vs. {}</b>".format(cat1,cat2),
  template="plotly_white",
  hover_data=['TEAM_NAME', 'GP', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'OREB',
       'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS'], 
  height=800, 
  width=1200,
  )

fig_scatter_playoffs.update_layout(
    title_x=0.5,
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=20)),
    xaxis = dict(tickfont = dict(size=20)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig_scatter_playoffs.update_traces(marker={'size': 10})

fig_scatter_playoffs

In [ ]:
fig_scatter_playoffs.write_html("/Users/rdesh723/statpulse-html/plots/Team Box Scores/{}/Playoffs All Time Team {} vs. {}.html".format(save_year,cat1_playoffs,cat2_playoffs),full_html=False, include_plotlyjs='cdn')
# fig_scatter_playoffs.write_html("/Users/rdesh723/statpulse-html/plots/Team Box Scores/{}/Team {} vs. {} For The {} Playoffs.html".format(save_year,cat1_playoffs,cat2_playoffs, current_year_playoffs),full_html=False, include_plotlyjs='cdn')

In [ ]:
#MULTI METRIC PLOTS TOGETHER

In [138]:
sort_categories_playoffs=['PTS', 'REB','DREB', 'OREB', 'AST', 'STL', 'BLK','TOV', 'PF', 'FGM', 'FGA', 'FG3M',
       'FG3A', 'FGMX', 'FG3MX', 'FTM',  'FTA','MIN']

In [48]:
#TRENDED

In [49]:
for i in sort_categories_playoffs:
    df_totals_playoffs['RK_Category']=df_totals_playoffs[i].rank(method="dense", ascending=False)
    df_totals_playoffs['RK_Category']=df_totals_playoffs['RK_Category'].astype(int)
    top_playoffs=df_totals_playoffs.sort_values(by=i, ascending=False).head(10)
    names_playoffs=top_playoffs.TEAM_NAME.to_list()
    df_top_playoffs=df_bq2_playoffs[df_bq2_playoffs['TEAM_NAME'].isin(names)]

    fig_multi_playoffs = px.scatter(
    df_top, 
    x="Game Number", 
    y="Total {}".format(i),
    color='TEAM_ABBREVIATION',
color_discrete_map=team_color_dict,     
    title="<b>All Time Trended Team {} vs. Game Number</b>".format(i),
    template="plotly_white",
    hover_data=['TEAM_NAME','Total PTS', 'Total REB',
       'Total DREB', 'Total OREB', 'Total AST', 'Total STL', 'Total BLK',
       'Total TOV', 'Total PF', 'Total FGM', 'Total FGA', 'Total FG3M',
       'Total FG3A', 'Total FGMX', 'Total FG3MX', 'Total FTM', 'Total FTA',
       'Total MIN'
        #          'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'OREB',
        # 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS', 'Season',
        # 'Game Number'
        ], 
    height=800, 
    width=1200,
    )

    fig_multi_playoffs.update_layout(
        title_x=0.5,
        hoverlabel=dict(
            bgcolor="white",
            font_size=16,
            font_family="PT Sans Narrow"
        ),
        yaxis = dict(tickfont = dict(size=20)),
        xaxis = dict(tickfont = dict(size=20)),
        font=dict(
            family="PT Sans Narrow",
            size=14,
            color="Black"
        ),
        title_font_family="PT Sans Narrow"
    )
    fig_multi_playoffs.update_traces(marker={'size': 3})
    fig_multi_playoffs.write_html("/Users/rdesh723/statpulse-html/plots/Team Box Scores/{}/Playoffs All Time Trended Team {} vs. Game Number.html".format(save_year,i),full_html=False, include_plotlyjs='cdn')
    # fig_multi_playoffs.write_html("/Users/rdesh723/statpulse-html/plots/Team Box Scores/{}/Trended Team {} vs. Game Number For The {} Playoffs.html".format(save_year,i, current_year_playoffs),full_html=False, include_plotlyjs='cdn')    

In [ ]:
#POLAR

In [139]:
for i in sort_categories_playoffs:
    df_totals_playoffs['RK_Category']=df_totals_playoffs[i].rank(method="dense", ascending=False)
    df_totals_playoffs['RK_Category']=df_totals_playoffs['RK_Category'].astype(int)
    top_playoffs=df_totals_playoffs.sort_values(by=i, ascending=False).head(10)
    names_playoffs=top_playoffs.TEAM_NAME.to_list()
    df_top_playoffs=df_bq2_playoffs[df_bq2_playoffs['TEAM_NAME'].isin(names)]
    
    polar_multi_playoffs=px.bar_polar(
    top_playoffs, 
    r=i, 
    theta="TEAM_NAME", 
    color="TEAM_ABBREVIATION",
    template='ggplot2',
    color_discrete_map=team_color_dict,     
    # title="All Time Leaders In Total {}".format(i),
    title="Leaders In Total Team {} For The {} Playoffs.html".format(i, current_year_playoffs),
    hover_data=['TEAM_NAME', 'MIN', 'FGM', 'FGMX', 'FGA', 'FG_PCT', 'FG2M', 'FG2A',
        'FG2_PCT', 'FG3M', 'FG3MX', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'OREB',
        'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS',
        'RK_Category'],
    height=800, width=1200,
                    )
    polar_multi_playoffs.update_layout(
        title_x=0.5,
        hoverlabel=dict(
            bgcolor="white",
            font_size=16,
            font_family="PT Sans Narrow"
        ),
        yaxis = dict(tickfont = dict(size=20)),
        xaxis = dict(tickfont = dict(size=20)),
        font=dict(
            family="PT Sans Narrow",
            size=14,
            color="Black"
        ),
        title_font_family="PT Sans Narrow"
    )
    # polar_multi_playoffs.write_html("/Users/rdesh723/statpulse-html/plots/Team Box Scores/{}/Playoffs All Time Leaders In Total Team {}.html".format(save_year,i),full_html=False, include_plotlyjs='cdn')
    polar_multi_playoffs.write_html("/Users/rdesh723/statpulse-html/plots/Team Box Scores/{}/Leaders In Total Team {} For The {} Playoffs.html".format(save_year,i, current_year_playoffs),full_html=False, include_plotlyjs='cdn')    